In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
import seaborn as sns
import math

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
X, y = mnist["data"], mnist["target"]

In [4]:
y = y.astype(np.uint8)

In [5]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [6]:
enc.fit(y[:,np.newaxis])

OneHotEncoder()

In [7]:
Y = enc.transform(y[:,np.newaxis]).toarray()

In [8]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], Y[:60000], Y[60000:]

In [9]:
X_train = X_train / 255
X_test = X_test / 255

In [10]:
X_train.shape

(60000, 784)

In [11]:
X_train, X_valid, y_train, y_valid = X_train[:50000], X_train[50000:], y_train[:50000], y_train[50000:]

In [12]:
X_valid = np.hstack((np.ones((np.size(X_valid, 0),1)),X_valid))

In [13]:
X_valid.shape

(1000, 785)

In [14]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [15]:
def softmax(X, W):
    K = np.size(W, 1)
    A = np.exp(X @ W)
    B = np.diag(1 / (np.reshape(A @ np.ones((K,1)), -1))) # reshape(, -1) -> (10, 1) -> (10)
    Y = B @ A
    return Y

In [16]:
def compute_cost(X, T, W, L2_Regular):
    epsilon = 1e-5
    N = len(T)
    K = np.size(T, 1)
    # 기존 cost + L2
    cost = - (1/N) * np.ones((1,N)) @ ((np.multiply(np.log(softmax(X, W) + epsilon), T))) @ np.ones((K,1)) + L2_Regular
    #cost = - (1/N) * np.ones((1,N)) @ (np.multiply(np.log(softmax(X, W) + epsilon), T)) @ np.ones((K,1))
    return cost

In [17]:
def predict(X, W):
    return np.argmax((X @ W), axis=1)

In [18]:
'''
def batch_gd(X, T, W, learning_rate, iterations, batch_size):
    N = len(T)
    cost_history = np.zeros((iterations,1))
    shuffled_indices = np.random.permutation(N)
    X_shuffled = X[shuffled_indices]
    T_shuffled = T[shuffled_indices]
    
    for i in range(iterations):
        j = i % N
        X_batch = X_shuffled[j:j+batch_size]
        T_batch = T_shuffled[j:j+batch_size]
        # batch가 epoch 경계를 넘어가는 경우, 앞 부분으로 채워줌
        if X_batch.shape[0] < batch_size:
            X_batch = np.vstack((X_batch, X_shuffled[:(batch_size - X_batch.shape[0])]))
            T_batch = np.vstack((T_batch, T_shuffled[:(batch_size - T_batch.shape[0])]))
        
        # lambda 적용..?
        valid = predict(X_valid, W)
        
        #lamda = float(sum(valid == np.argmax(y_valid, axis=1)))/ float(len(y_valid))
        #lamda = (1 - float(sum(valid == np.argmax(y_valid, axis=1)))/ float(len(y_valid))) * 0.001
        lamda = 0
        # L2 구하기: W 제곱의 합 * lambda/2n
        
        L2_Regular = (lamda/(2 * len(X))) * np.ones((1, len(W))) @ (W ** 2) @ np.ones((np.size(W, 1), 1))
        L2_norm_d = W * 2 * lamda # L2 norm의 미분값
        
        W = W - (learning_rate/batch_size) * (X_batch.T @ (softmax(X_batch, W) - T_batch) + L2_norm_d)
        # W = W - (learning_rate/batch_size) * (X_batch.T @ (softmax(X_batch, W) - T_batch)) 
        cost_history[i] = compute_cost(X_batch, T_batch, W, L2_Regular)
        if i % 1000 == 0:
            #print(cost_history[i][0])
            print("lamda", lamda)
            print("L2_norm_d", np.ones((1, len(W))) @ W ** 2 @ np.ones((np.size(W, 1), 1)))
            print("-------------")

    return (cost_history, W)
'''

'\ndef batch_gd(X, T, W, learning_rate, iterations, batch_size):\n    N = len(T)\n    cost_history = np.zeros((iterations,1))\n    shuffled_indices = np.random.permutation(N)\n    X_shuffled = X[shuffled_indices]\n    T_shuffled = T[shuffled_indices]\n    \n    for i in range(iterations):\n        j = i % N\n        X_batch = X_shuffled[j:j+batch_size]\n        T_batch = T_shuffled[j:j+batch_size]\n        # batch가 epoch 경계를 넘어가는 경우, 앞 부분으로 채워줌\n        if X_batch.shape[0] < batch_size:\n            X_batch = np.vstack((X_batch, X_shuffled[:(batch_size - X_batch.shape[0])]))\n            T_batch = np.vstack((T_batch, T_shuffled[:(batch_size - T_batch.shape[0])]))\n        \n        # lambda 적용..?\n        valid = predict(X_valid, W)\n        \n        #lamda = float(sum(valid == np.argmax(y_valid, axis=1)))/ float(len(y_valid))\n        #lamda = (1 - float(sum(valid == np.argmax(y_valid, axis=1)))/ float(len(y_valid))) * 0.001\n        lamda = 0\n        # L2 구하기: W 제곱의 합 * lambda/2n\n

In [47]:
def batch_gd(X, T, learning_rate, iterations, batch_size):
    N = len(T)
    cost_history = np.zeros((iterations,1))
    shuffled_indices = np.random.permutation(N)
    X_shuffled = X[shuffled_indices]
    T_shuffled = T[shuffled_indices]
    max_acc = 0
    max_lamda = 0
    x_acc = 0
    max_W = np.zeros((np.size(X, 1), np.size(T, 1)))
    lamda = 0
    
    for k in range(101):
        W = np.zeros((np.size(X, 1), np.size(T, 1)))
        cnt1 = 0
        cnt2 = 0
        print("lambda", lamda)
        for i in range(iterations):
            j = i % N
            X_batch = X_shuffled[j:j+batch_size]
            T_batch = T_shuffled[j:j+batch_size]
            # batch가 epoch 경계를 넘어가는 경우, 앞 부분으로 채워줌
            if X_batch.shape[0] < batch_size:
                X_batch = np.vstack((X_batch, X_shuffled[:(batch_size - X_batch.shape[0])]))
                T_batch = np.vstack((T_batch, T_shuffled[:(batch_size - T_batch.shape[0])]))
            
            L2_Regular = lamda * math.sqrt(np.ones((1, len(W))) @ (W ** 2) @ np.ones((np.size(W, 1), 1)))

            W = W * (1 - 2 * learning_rate/batch_size * lamda) - (learning_rate/batch_size) * (X_batch.T @ (softmax(X_batch, W) - T_batch)) 
            cost_history[i] = compute_cost(X_batch, T_batch, W, L2_Regular)
            #if i % 1000 == 0:
                #print("{} %...".format(i/200))

        y_pred = predict(X_valid, W)
        score = float(sum(y_pred == np.argmax(y_valid, axis=1)))/ float(len(y_valid))
        print("Sum W^2", np.ones((1, len(W))) @ W ** 2 @ np.ones((np.size(W, 1), 1)))
        print("acc", score)
        if max_acc < score:
            max_W = W
            max_acc = score
            if lamda != 0:
                lamda -= (1 - score) * 0.05
            print("max change!!")
        else:
            if x_acc > score:
                lamda -= (1 - score) * 0.02
            else:
                lamda += (1 - score) * 0.05
        print("---------------------------------------")
        if max_acc - score > 0.04:
            print("early stopping...")
            break
        x_acc = score
            
    print('max lambda: ',max_lamda)
    print('max acc: ',max_acc)
    return (cost_history, max_W)

In [48]:
X = np.hstack((np.ones((np.size(X_train, 0),1)),X_train))
T = y_train

K = np.size(T, 1)
M = np.size(X, 1)
W = np.zeros((M,K))

iterations = 20000
learning_rate = 0.01

initial_cost = compute_cost(X, T, W, 0)

print("Initial Cost is: {} \n".format(initial_cost[0][0]))
(cost_history, W_optimal) = batch_gd(X, T, learning_rate, iterations, 64)

Initial Cost is: 2.3024850979937126 

lambda 0
Sum W^2 [[80.24403349]]
acc 0.937
max change!!
---------------------------------------
lambda 0
Sum W^2 [[80.24403349]]
acc 0.937
---------------------------------------
lambda 0.0031499999999999974
Sum W^2 [[78.99902183]]
acc 0.937
---------------------------------------
lambda 0.006299999999999995
Sum W^2 [[77.78798384]]
acc 0.937
---------------------------------------
lambda 0.009449999999999993
Sum W^2 [[76.60979684]]
acc 0.937
---------------------------------------
lambda 0.01259999999999999
Sum W^2 [[75.46337792]]
acc 0.937
---------------------------------------
lambda 0.015749999999999986
Sum W^2 [[74.34768248]]
acc 0.936
---------------------------------------
lambda 0.014469999999999986
Sum W^2 [[74.79740411]]
acc 0.936
---------------------------------------
lambda 0.017669999999999984
Sum W^2 [[73.68227866]]
acc 0.936
---------------------------------------
lambda 0.020869999999999982
Sum W^2 [[72.59720441]]
acc 0.936
-------

Sum W^2 [[39.00987933]]
acc 0.935
---------------------------------------
lambda 0.21123999999999998
Sum W^2 [[38.71221076]]
acc 0.934
---------------------------------------
lambda 0.20992
Sum W^2 [[38.83248153]]
acc 0.934
---------------------------------------
lambda 0.21322
Sum W^2 [[38.53339462]]
acc 0.933
---------------------------------------
lambda 0.21187999999999999
Sum W^2 [[38.65420398]]
acc 0.933
---------------------------------------
lambda 0.21522999999999998
Sum W^2 [[38.35379398]]
acc 0.933
---------------------------------------
lambda 0.21857999999999997
Sum W^2 [[38.058688]]
acc 0.933
---------------------------------------
lambda 0.22192999999999996
Sum W^2 [[37.76875031]]
acc 0.933
---------------------------------------
lambda 0.22527999999999995
Sum W^2 [[37.48384946]]
acc 0.933
---------------------------------------
lambda 0.22862999999999994
Sum W^2 [[37.20385815]]
acc 0.933
---------------------------------------
lambda 0.23197999999999994
Sum W^2 [[36.928

In [49]:
## Accuracy
X_ = np.hstack((np.ones((np.size(X_test, 0),1)),X_test))
T_ = y_test
y_pred = predict(X_, W_optimal)
score = float(sum(y_pred == np.argmax(T_, axis=1)))/ float(len(y_test))

print(score)

0.9039
